In [11]:

import rasterio as rs
from rasterio import plot as pl
import requests
from shapely.geometry import Polygon
import numpy as np

import plotly.graph_objects as go
from rasterio.mask import mask



In [2]:
#using the geoPunt-API to request the adresses 
#I've this code from my peers, it is not mine but I understand it!

def get_coordinates(address: str):
    req = requests.get(f"https://loc.geopunt.be/v4/Location?q={address}").json()
    
    #putting everything in to a dictionary
    info = {'address' : address, 
                'x_value' : req['LocationResult'][0]['Location']['X_Lambert72'],
                'y_value' : req['LocationResult'][0]['Location']['Y_Lambert72'],
                'street' : req['LocationResult'][0]['Thoroughfarename'],
                'house_number' : req['LocationResult'][0]['Housenumber'], 
                'postcode': req['LocationResult'][0]['Zipcode'], 
                'municipality' : req['LocationResult'][0]['Municipality']}
    
    #using this API from the government of Belgium, we can get the formal details from a given adress
    detail = requests.get("https://api.basisregisters.vlaanderen.be/v1/adresmatch", 
                          params={"postcode": info['postcode'], 
                                  "straatnaam": info['street'],
                                  "huisnummer": info['house_number']}).json()
    building = requests.get(detail['adresMatches'][0]['adresseerbareObjecten'][0]['detail']).json()
    build = requests.get(building['gebouw']['detail']).json()
    info['polygon'] = [build['geometriePolygoon']['polygon']]
    return info['polygon'][0]['coordinates'][0]

In [7]:
"""
this function  will do the file management, the data has to be in this directory.

"""

def getting_CHM(coord):
    
    dsm_file = 'DHMVIIDSMRAS1m_k15.tif'
    dtm_file = 'DHMVIIDTMRAS1m_k15.tif'
    
    
    
    img_dsm = rs.open('dsm15.tif') #dsm_file has to be the argument
    dsm = img_dsm.read(1)
    
    img_dtm = rs.open('dtm15.tif')
    dtm = img_dtm.read(1)
    
    poly = Polygon(coord)
    
    #masking dsm values
    x_coords_dsm, y_coords_dsm = mask(dataset=img_dsm, shapes=[poly],crop=True, nodata=0,filled=True,indexes=1)

    #masking dtm values
    x_coords_dtm, y_coords_dtm = mask(dataset=img_dtm, shapes=[poly],crop=True, nodata=0,filled=True,indexes=1)
    chm = x_coords_dsm - x_coords_dtm
    
    return chm

In [8]:
def plot():
    adres = 'Koningin Astridplein 27'
    coord = get_coordinates(adres)
    chm = getting_CHM(coord)
    
    x = np.arange(chm.shape[0])
    y = np.arange(chm.shape[1])

    figuur= go.Figure(data=[go.Surface(x=x,y=y,z=chm)])
    
    figuur.show()

In [9]:
plot()